In [1]:
#use household income api

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import time
from pprint import pprint
from us import states
import gmaps
 

# Census API Key
from config import (api_key, weather_api, gkey)




In [3]:
file_one = "./UnemploymentReport.csv"

income_df = pd.read_csv(file_one)
house_income = income_df[["Name", "Median Household Income (2016)"]]
house_income = house_income.dropna(how='any')


In [4]:
cities = house_income["Name"]
g_lat = []
g_lon= []
g_name = []

for city in cities:
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(city, gkey)
    geo_data = requests.get(target_url).json()
    g_name.append(city)
    g_lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
    g_lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
        

# Print the json


In [5]:
g_dict = {"Name": g_name,
          "g_lat": g_lat,
          "g_lon": g_lon}

g_df = pd.DataFrame(g_dict)


In [23]:
final_table = pd.merge(house_income, g_df, on="Name", how="left")
final_table

final_table.to_csv("./final_household_income.csv", index=False, header = True)

In [7]:
gmaps.configure(api_key=gkey)

In [24]:
locations = final_table[["g_lat", "g_lon"]].astype(float)

household_income = final_table["Median Household Income (2016)"].astype(float)



In [27]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=household_income, 
                                 dissipating=False, max_intensity=100000,
                                 point_radius = 1)


heat_layer.dissipating = False
heat_layer.max_intensity = 100000
heat_layer.point_radius = .5

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))